In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import torch
import os
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [13]:
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()

imagenet_classes = ResNet50_Weights.IMAGENET1K_V1.meta["categories"]
print("ResNet-50 loaded")


ResNet-50 loaded


In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [15]:
valid_cat_keywords = [
    "cat", "tabby", "tiger", "persian", "egyptian",
    "siamese", "lynx", "cheetah", "leopard", "jaguar", "lion"
]


In [17]:
DATASET_PATH = "/content/drive/MyDrive/Gen_Ai_Lab/Cat_Species"


import os
print("Species folders:", os.listdir(DATASET_PATH)[:5])


Species folders: ['Abyssinian', 'American_Bobtail', 'checkpoint.json']


In [18]:
correct = 0
total = 0

for species in os.listdir(DATASET_PATH):
    species_path = os.path.join(DATASET_PATH, species)
    if not os.path.isdir(species_path):
        continue

    for img_name in os.listdir(species_path):
        if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(species_path, img_name)

        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            pred_idx = output.argmax(dim=1).item()

        label = imagenet_classes[pred_idx].lower()
        print(f"{img_name} → {label}")

        if any(k in label for k in valid_cat_keywords):
            correct += 1

        total += 1

accuracy = (correct / total) * 100
print(f"\nResNet Proxy Accuracy (Cat Detection): {accuracy:.2f}%")


Abyssinian_1.png → egyptian cat
Abyssinian_2.png → egyptian cat
Abyssinian_3.png → egyptian cat
Abyssinian_4.png → egyptian cat
Abyssinian_5.png → egyptian cat
Abyssinian_6.png → egyptian cat
Abyssinian_7.png → egyptian cat
Abyssinian_8.png → egyptian cat
Abyssinian_9.png → egyptian cat
Abyssinian_10.png → egyptian cat
American_Bobtail_1.png → tabby
American_Bobtail_2.png → tabby

ResNet Proxy Accuracy (Cat Detection): 100.00%
